In [954]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1328]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import normalize as sklearn_normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from passauth import data
from passauth import utils
from passauth import authenticator

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn import svm

import warnings
warnings.filterwarnings('ignore')

In [5]:
mypath = '/Users/admin/Downloads/entries/data/'
user_data = data.UserData(mypath)

## Plotting user behavior

When we try to draw a picture of the user behavior, we can clearly see a unique pattern to two users.


In [ ]:

# Plot user 1
user1 = user_data.getUserData(10)
utils.plotUserMouseScatterMultiple(user1, n=5)

In [ ]:
user1 = user_data.getUserData(6)
utils.plotUserMouseScatterMultiple(user1, n=5)

## Time for some Machine Learning

'After some failed attempts at one-shot learning using deep NNs'

In [ ]:
mypath = '/Users/admin/Downloads/entries/data/'

In [47]:

def rf_ova_classifier(dataX, dataY, test_size=0.3, cross_validate=0):
    X_train, X_test, y_train, y_test = train_test_split(
    dataX, dataY, test_size=test_size, random_state=30)
    
    if cross_validate == 1:
        X_test = X_train
        y_test = y_train
    
    for each in range(len(np.unique(y_train))):
        _RF = RandomForestClassifier(n_estimators=100)
        print(f'train {list(y_train).count(each)}')
        _RF.fit(X_train, np.where(y_train != each, -1, each))
        res = _RF.predict(X_test)
        
        print(f'Y-test {list(y_test).count(each)}', end='')
        
        print(f'result {list(res).count(each)}', end='')

      
        
        print(f"Score for User {each}:\nAccuracy: {_RF.score(X_test, np.where(y_test != each, -1, each))}")

        FAR = [(real != each) and (guess == each) for real,guess in zip(y_test, res)].count(True)
        FRR = [(real == each) and (guess != each) for real,guess in zip(y_test, res)].count(True)
        

        print(f"FAR: {FAR} / {list(np.where(y_test != each, -1, 0)).count(-1)}", end=' ')
        print(f"FRR: {FRR} / {list(np.where(y_test == each, each, -1)).count(each)}\n")
        



def generic_ova_discriminator(dataX, dataY, test_size=0.3, cross_validate=0, method=svm.SVC, params={}, verbose=1):

    X_train, X_test, y_train, y_test = train_test_split(
    dataX, dataY, test_size=test_size, random_state=30)
    
    if cross_validate == 1:
        X_test = X_train
        y_test = y_train
    

    SVM = method(**params)
    SVM.fit(X_train, y_train)

    correct = [0,0]
    incorrect = [0,0]
    for each in range(len(np.unique(y_train))):
        
        #params['class_wight'] = {each:10}
        _SVM = method(**params)

        _SVM.fit(X_train, np.where(y_train != each, -1, each))
        res = _SVM.predict(X_test)
        
        FAR = [(real != each) and (guess == each) for real,guess in zip(y_test, res)].count(True)
        FRR = [(real == each) and (guess != each) for real,guess in zip(y_test, res)].count(True)
        
        correct[0] += FAR #false correct
        correct[1] += list(np.where(y_test != each, -1, 0)).count(-1)
        
        incorrect[0] += FRR #false rejection
        incorrect[1] += list(y_test).count(each)
        
        if verbose:
            print(f'Y-test- {list(y_test).count(each)}', end='')
            print(f' result- {list(res).count(each)}', end='')

            print(f"FAR: {FAR} / {list(np.where(y_test != each, -1, 0)).count(-1)}", end=' ')
            print(f"FRR: {FRR} / {list(np.where(y_test == each, each, -1)).count(each)}\n")
      
    print(f"FAR = {correct[0]/correct[1]*100} AND FRR = {incorrect[0]/incorrect[1]*100} ")
        


def k_means_discriminator(dataX, dataY, test_size=0.3, cross_validate=0):

    X_train, X_test, y_train, y_test = train_test_split(
    dataX, dataY, test_size=test_size, random_state=30)
    
    
    if cross_validate == 1:
        X_test = X_train
        y_test = y_train
    


    for each in range(len(np.unique(y_train))):

        _k_means = KMeans(n_clusters=2)

        _k_means.fit(X_train, np.where(y_train != each, -1, each))
        res = _k_means.predict(X_test)
        
        print(f'Y-test {list(y_test).count(each)}', end='')
        
        print(f'result {list(res).count(each)}', end='')

        FAR = [(real != each) and (guess == each) for real,guess in zip(y_test, res)].count(True)
        FRR = [(real == each) and (guess != each) for real,guess in zip(y_test, res)].count(True)
        

        print(f"FAR: {FAR} / {list(np.where(y_test != each, -1, 0)).count(-1)}", end=' ')
        print(f"FRR: {FRR} / {list(np.where(y_test == each, each, -1)).count(each)}\n")
        


## Mouse Dynamics




In [804]:
k_params = {'min_entries':20}
m_params = {'min_entries':20, 'padding':200}

Authenticator = authenticator.Authenticator(path=mypath, k_params=k_params, m_params=m_params)
# Raw data with (x, y, time)
dataX, dataY = Authenticator.get_mouse_data(Authenticator.mouse_data, method=1)

In [48]:
results = {}
generic_ova_discriminator(dataX, dataY, method=svm.LinearSVC, params={'loss':'squared_hinge'},verbose=0)

/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


FAR = 12.749445676274945 AND FRR = 74.39024390243902 


/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
generic_ova_discriminator(dataX, dataY, method=svm.SVC, params={'kernel':'linear'},verbose=0)

In [ ]:
# Raw data with (x, y)
dataX, dataY = Authenticator.get_mouse_data(Authenticator.mouse_data, method=1)
generic_ova_discriminator(dataX, dataY, method=svm.SVC, params={'kernel':'linear'},verbose=0)

In [ ]:
# Transform dataX to include only n-points
_dataX = [each[0::10] for each in dataX]

In [ ]:
generic_ova_discriminator(_dataX, dataY, method=svm.SVC, params={'kernel':'linear'},verbose=0)

In [ ]:
generic_ova_discriminator(_dataX, dataY, method=svm.SVC, params={'kernel':'linear'},verbose=0)

In [ ]:
generic_ova_discriminator(sklearn_normalize(_dataX), dataY, method=svm.SVC, params={'kernel':'linear'},verbose=0)

In [ ]:
generic_ova_discriminator(sklearn_normalize(_dataX)*1000, dataY, method=svm.SVC, params={'kernel':'linear', 'degree':5},verbose=0)

In [ ]:
# Transform dataX to include only x cordinates of n-points
_dataX = [each[1::2] for each in _dataX]

In [ ]:
# Transform dataX to include only n-points
generic_ova_discriminator(sklearn_normalize(np.cumsum(_dataX, axis=1), norm='l2', axis=0)*5000, dataY, method=svm.SVC, params={'kernel':'linear', 'degree':5},verbose=0)

In [ ]:
generic_ova_discriminator(np.cumsum(_dataX, axis=0), dataY, method=svm.SVC, params={'kernel':'linear', 'degree':5},verbose=0)

In [ ]:
#only 10th points
generic_ova_discriminator(_dataX, dataY, method=RandomForestClassifier, params={'n_estimators':20},verbose=0)

In [ ]:
# Transform dataX to include only x cordinates of n-points
_dataX = [each[0::2] for each in [each[0::1] for each in dataX]]
generic_ova_discriminator(sklearn_normalize(_dataX, norm='l2', axis=0)*1000, dataY, method=GaussianNB, verbose=0)

In [ ]:
#PCA

X_train, X_test, y_train, y_test = train_test_split(
    dataX, dataY, test_size=0.3, random_state=30)

In [ ]:
#for each in range(len(np.unique(y_train))):
pca = PCA(10)
each = 0
projected = pca.fit_transform(X_train[y_train == each])


In [ ]:
classifier = svm.OneClassSVM()


In [ ]:
classifier.fit(pca.transform(X_train) * 100, np.where(y_train != each, -1, each))

In [ ]:
test_projected = pca.transform(X_test)
#classifier.score(test_projected * 100,  np.where(y_test != each, -1, each)) 

In [ ]:
 np.where(y_test != each, -1, each)[classifier.predict(test_projected) == each]

In [ ]:
list(np.where(y_test != each, -1, each)).count(each)

## Feature Engineering

1. Calculate the center.
2. Calculate the directions.
3. Bag the events into 4-8 directions.

In [803]:
dataX, dataY = Authenticator.get_mouse_data(Authenticator.mouse_data, method=1)

In [69]:
transformed_dataX = []
for each in dataX:
    transformed_dataX.append((each[::2] - 500) * (each[1::2] - 250))
transformed_dataX = np.array(transformed_dataX)

In [81]:
generic_ova_discriminator(_dataX, dataY, method=svm.SVC, params={'kernel': 'linear', 'max_iter':1000})

Y-test- 4 result- 51FAR: 49 / 78 FRR: 2 / 4

Y-test- 8 result- 9FAR: 8 / 74 FRR: 7 / 8

Y-test- 9 result- 32FAR: 26 / 73 FRR: 3 / 9

Y-test- 5 result- 15FAR: 14 / 77 FRR: 4 / 5

Y-test- 10 result- 10FAR: 8 / 72 FRR: 8 / 10

Y-test- 7 result- 50FAR: 44 / 75 FRR: 1 / 7

Y-test- 7 result- 33FAR: 29 / 75 FRR: 3 / 7

Y-test- 3 result- 70FAR: 67 / 79 FRR: 0 / 3

Y-test- 6 result- 45FAR: 41 / 76 FRR: 2 / 6

Y-test- 8 result- 34FAR: 33 / 74 FRR: 7 / 8

Y-test- 9 result- 59FAR: 51 / 73 FRR: 1 / 9

Y-test- 6 result- 42FAR: 42 / 76 FRR: 6 / 6

FAR = 45.67627494456763 AND FRR = 53.65853658536586 


/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.

In [ ]:
classifier = svm.SVC(kernel='linear')

In [ ]:
classifier.fit(transformed_dataX, dataY)

In [ ]:
classifier.score(transformed_dataX, dataY)

# Calculate the Quadrants:
```
   \  8 |  1 /         
     \  |  /  2
_______\|/______
   6   /|\  3
     /  |4 \
   /    |    \
```

# Keystroke and mouse dynamics combined

In [309]:
import numpy as np
from sklearn.preprocessing import StandardScaler


#anglify(Authenticator.mouse_data[0][0])
#anglify(Authenticator.mouse_data[1][0])

In [1411]:
k_params = {'min_entries':15}
m_params = {'min_entries':15, 'padding':100}

Authenticator = authenticator.Authenticator(path=mypath, k_params=k_params, m_params=m_params)
# Raw data with (x, y, time)
dataX, dataY = Authenticator.get_mouse_data(Authenticator.mouse_data, method=1)

(285,)

In [1413]:
def score(user, predicted, actual, verbose = 0):

    false_positive = [(real != user) and (guess == user) for real,guess in zip(actual, predicted)].count(True)
    false_negative = [(real == user) and (guess != user) for real,guess in zip(actual, predicted)].count(True)

    total_positive = list(actual).count(user)
    total_negative = list(actual).count(-1)
    
    assert total_negative != 0 
    assert total_positive !=0
    far = false_positive/total_negative
    frr = false_negative/total_positive

    if verbose:
        print(f'test-positive: {total_positive}', end=' AND ')
        print(f'test-negative: {total_negative}', end='\n')

        print(f"FAR: {false_positive} / {total_negative} = {far*100}%", end=' ')
        print(f"FRR: {false_negative} / {total_positive} = {frr*100}%", end=' \n')

    return far, frr

def n_elems(arr, n):
    return np.array([each[::n] for each in arr])

def get_angle(p0, p1=np.array([0,0]), p2=None):
    ''' compute angle (in degrees) for p0p1p2 corner
    Inputs:
        p0,p1,p2 - points in the form of [x,y]
    '''
    if p2 is None:
        p2 = p1 + np.array([1, 0])
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return np.degrees(angle)

def anglify(arr, if_cumsum=0,):
    angle = arr[0][0:2]
    if if_cumsum:
        return np.cumsum([get_angle(each[0:2], p2=angle) for each in arr])
    return np.array([get_angle(each[0:2], p2=angle) for each in arr])

def anglified(arr, if_cumsum=0,):
    return [anglify(each, if_cumsum=if_cumsum) for each in arr]




def custom_train_test(user, 
                      X_train, X_test, y_train, y_test, 
                      cross_validate=0, 
                      method=svm.SVC, 
                      params={}, 
                      verbose=1, 
                      return_type='analysis'):
    
    _SVM = method(**params)
    _SVM.fit(X_train, np.where(y_train != user, -1, user))
    res = _SVM.predict(X_test)

    test_score = score(user, res, y_test, verbose=verbose)

    analysis = {'res': res, 'far': test_score[0] , 'frr': test_score[1] } 

    if cross_validate == 1:
        analysis ['cross_validate_far'], analysis ['cross_validate_frr'] = score(user, _SVM.predict(X_train), y_train)
    
    if return_type == 'model':
        return _SVM
    elif return_type == 'analysis':
        return analysis



    
class User:
    def __init__(self, user, dataX, dataY, kdataX, kdataY, test_size=0.3, raw_mouse_data=[], random_state=30):
        self.user = user
        self.dataX = dataX
        self.dataY = dataY
        self.kdataX = kdataX
        self.kdataY = kdataY
        self.random_state = random_state
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
        dataX, dataY, test_size=test_size, random_state=self.random_state)
        
        self.X_train_k, self.X_test_k, self.y_train_k, self.y_test_k = train_test_split(
        self.kdataX, self.kdataY, test_size=test_size, random_state=self.random_state)
 
        self.y_train = np.where(self.y_train != user, -1, 0)
        self.y_test = np.where(self.y_test != user, -1, 0)
        
        self.y_train_k = np.where(self.y_train_k != user, -1, 0)
        self.y_test_k = np.where(self.y_test_k != user, -1, 0)
        
        self.X_raw_mouse = np.array([x[0] for x in raw_mouse_data])
        self.y_raw_mouse = np.array([x[1] for x in raw_mouse_data])
        
        self.y_raw_mouse = np.where(self.y_raw_mouse != self.user, -1, 0)

        self.X_train_raw_mouse, self.X_test_raw_mouse, self.y_train_raw_mouse, self.y_test_raw_mouse = train_test_split(
        self.X_raw_mouse, self.y_raw_mouse, test_size=test_size, random_state=self.random_state)

        #set the best model configurations
        self.best_models_keyboard = []
        self.best_models_keyboard_o = [] #to store the classifier
        self.best_models_mouse = []
        self.best_models_mouse_o = []


    def score(self, predicted, actual, verbose = 0):
        return score(0, predicted, actual, verbose)

      
        
    def train_test(self, 
                   cross_validate=0, 
                   method=[svm.SVC], 
                   params={}, 
                   verbose=1, 
                   transformation=(0,0), 
                   scale=0, 
                   pca=0,
                   return_type='analysis'):
        
        X_train = []
        X_test = []
        
        if transformation[0] == 1: #drop every element except n-th
            X_train = n_elems(self.X_train, transformation[1])
            X_test = n_elems(self.X_test, transformation[1])

        elif transformation[0] == 2: #calculate the angle
            X_train = n_elems(anglified(self.X_train_raw_mouse, if_cumsum=transformation[1]), transformation[2])
            X_test = n_elems(anglified(self.X_test_raw_mouse, if_cumsum=transformation[1]), transformation[2])
            

        else:
            X_train = self.X_train
            X_test = self.X_test
            
        if pca > 0:
            pca = PCA(n_components=pca, svd_solver='randomized', whiten=True).fit(self.X_train)
            X_train = pca.transform(self.X_train)
            X_test = pca.transform(self.X_test)
            
        if scale:
            scaler = StandardScaler()
            if X_train != []:
                X_train= scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)


            else:
                X_train= scaler.fit_transform(self.X_train)
                X_test = scaler.transform(self.X_test)



        return custom_train_test(0, X_train, X_test, self.y_train, self.y_test, 
                                cross_validate=cross_validate, 
                                 method=method, 
                                 params=params, 
                                 verbose=verbose, 
                                 return_type=return_type)
  
    def transform_keyboard_data(self, data, transformation=(0,0), pca=0, scale=0):
        
        if transformation[0] == 1: #drop every element except n-th
            data = n_elems(self.data, transformation[1])

            
        if pca > 0:
            pca = PCA(n_components=pca, svd_solver='randomized', whiten=True).fit(self.X_train_k)
            data = pca.transform(data)
            
        if scale:
            scaler = StandardScaler().fit(X_train)
            data = scaler.transform(data)

        return data



    def train_test_keyboard(self, 
                            cross_validate=0, 
                            method=[svm.SVC], 
                            params={}, 
                            verbose=1, 
                            transformation=(0,0), 
                            scale=0, 
                            pca=0,
                            return_type='analysis',
                            model=None):
        X_train = []
        X_test = []
        
        if transformation[0] == 1: #drop every element except n-th
            X_train = n_elems(self.X_train_k, transformation[1])
            X_test = n_elems(self.X_test_k, transformation[1])

     
        else:
            X_train = self.X_train_k
            X_test = self.X_test_k
            
        if pca > 0:
            pca = PCA(n_components=pca, svd_solver='randomized', whiten=True).fit(self.X_train_k)
            X_train = pca.transform(self.X_train_k)
            X_test = pca.transform(self.X_test_k)
            
        if scale:
            scaler = StandardScaler()
            if X_train != []:
                X_train= scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)


            else:
                X_train= scaler.fit_transform(self.X_train_k)
                X_test = scaler.transform(self.X_test_k)

        return custom_train_test(0, X_train, X_test, self.y_train, self.y_test, 
                          cross_validate=cross_validate, method=method, params=params, verbose=verbose, return_type=return_type)
  


    def pipeline_train_test(self, parameters, verbose=0, keyboard=0):
        res = []
        for index, each in enumerate(parameters):
            if verbose != 0:
                print(each)
            if keyboard != 0:
                result = dict(self.train_test_keyboard(**each, verbose=verbose))
            else:
                result = dict(self.train_test(**each, verbose=verbose))
            result['config'] = each
            result['index'] = index
            res.append(result)
        return res

    def set_training_models(self, kb, mouse, n=5):
        self.best_models_keyboard = [_['config'] for _ in kb][:5]
        self.best_models_mouse = [_['config'] for _ in mouse][:5]
        
    def train_training_models(self):
        assert len(self.best_models_keyboard) != 0 and len(self.best_models_mouse) != 0, 'first find the best models'
        
        self.best_models_keyboard_o = [self.train_test_keyboard(**params, return_type='model') for params in self.best_models_keyboard]
        self.best_models_mouse_o = [self.train_test(**params, return_type='model') for params in self.best_models_keyboard]

In [1397]:
pipeline = [
    # SVM kernel
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'degree':4, 'max_iter':5000}},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}, 'transformation': (1, 3)},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}, 'transformation': (2, 1, 3)},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}, 'transformation': (2, 1, 3), 'scale':1},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}, 'pca':5, 'scale': 1},
    
    # Gaussian
    {'method':GaussianNB},
    {'method':GaussianNB, 'transformation': (1, 3), 'pca':5},
    {'method':GaussianNB, 'transformation': (1, 5)},
    {'method':GaussianNB, 'transformation': (2, 1, 3)},
    {'method':GaussianNB, 'transformation': (2, 0, 5)},

    # Bernoulli
    {'method':BernoulliNB},
    {'method':BernoulliNB, 'transformation': (1, 3)},
    {'method':BernoulliNB, 'transformation': (1, 5)},
    {'method':BernoulliNB, 'transformation': (2, 1, 3)},
    {'method':BernoulliNB, 'transformation': (2, 0, 5)},
    
    # Random Forest
    {'method':RandomForestClassifier},
    {'method':RandomForestClassifier, 'params': {'max_features': 'sqrt'}},
    {'method':RandomForestClassifier, 'params': {'n_estimators': 10}},
    {'method':RandomForestClassifier, 'transformation': (1, 10)},
    {'method':RandomForestClassifier, 'transformation': (2, 1, 3)},
    {'method':RandomForestClassifier, 'transformation': (2, 1, 10)},
]


keyboard_pipeline = [
    # SVM kernel
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'degree':4, 'max_iter':5000}},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}, 'transformation': (1, 3)},
    {'method':svm.SVC, 'params': {'kernel': 'linear', 'max_iter':5000}, 'pca':5, 'scale': 1},
    
    # Gaussian
    {'method':GaussianNB},
    {'method':GaussianNB, 'transformation': (1, 3), 'pca':5},
    {'method':GaussianNB, 'transformation': (1, 5)},


    # Bernoulli
    {'method':BernoulliNB},
    {'method':BernoulliNB, 'transformation': (1, 3)},
    {'method':BernoulliNB, 'transformation': (1, 5)},
   
    # Random Forest
    {'method':RandomForestClassifier},
    {'method':RandomForestClassifier, 'params': {'max_features': 'sqrt'}},
    {'method':RandomForestClassifier, 'params': {'n_estimators': 10}},
    {'method':RandomForestClassifier, 'transformation': (1, 10)},
]



In [1415]:
users = set(each[1] for each in Authenticator.mouse_data)

In [1414]:
kdataX = []
kdataY = []
for each in Authenticator.keystroke:
    kdataX.append(utils.bin(each[0]+each[1]))
    kdataY.append(each[2])
kdataY = np.array(dataY)

In [1400]:
grand_result_mouse = {}
grand_result_keyboard = {}

user_objects = {}
for each in users:
    user_objects[each] = User(each, dataX, dataY, kdataX, kdataY, raw_mouse_data=Authenticator.mouse_data)
    grand_result_mouse[each] = user_objects[each].pipeline_train_test(pipeline)
    grand_result_keyboard[each] = user_objects[each].pipeline_train_test(keyboard_pipeline, keyboard=1)
    
    user_objects[each].set_training_models(optimize_models(grand_result_keyboard[each], 'balanced'), 
                                           optimize_models(grand_result_mouse[each], 'balanced'),
                                           n=5)


In [1401]:
def final_test(user):
    mouse = np.sum([_['res'] for _ in optimize_models(grand_result_mouse[user], 'balanced')[:5]], axis=0)
    keyboard = np.sum([_['res'] for _ in optimize_models(grand_result_keyboard[user], 'balanced')[:5]], axis=0)
    
    combined = mouse + keyboard
    
    mouse = np.where(mouse > -3, 0, -1)
    keyboard = np.where(keyboard > -3, 0, -1)
    combined = np.where(combined > -6, 0, -1)
    
    return (user_objects[user].score(mouse, user_objects[user].y_test),
            user_objects[user].score(keyboard, user_objects[user].y_test_k),
            user_objects[user].score(combined, user_objects[user].y_test)
           )
    #print('Mouse only score: ', user_objects[user].score(mouse, user_objects[user].y_test))
    #print('Keyboard only score: ', user_objects[user].score(keyboard, user_objects[user].y_test_k))
    #print('Combined score: ', user_objects[user].score(combined, user_objects[user].y_test))
    
mouse_score = [0, 0]
keyboard_score = [0, 0]
combined_score = [0, 0]

for each in users:
    result = final_test(each)
    mouse_score[0] += result[0][0]
    mouse_score[1] += result[0][1]
    keyboard_score[0] += result[1][0]
    keyboard_score[1] += result[1][1]
    combined_score[0] += result[2][0]
    combined_score[1] += result[2][1]


In [1402]:
print('Mouse: ', mouse_score[0]/len(users), mouse_score[1]/len(users))
print('Keyboard: ', keyboard_score[0]/len(users), keyboard_score[1]/len(users))
print('Combined: ', combined_score[0]/len(users), combined_score[1]/len(users))

Mouse:  0.051467717533847476 0.6648351648351648
Keyboard:  0.15129839931695588 0.38986568986568987
Combined:  0.04352127019883322 0.5078144078144079


In [1372]:
def other_what(what):
    if what == 'frr':
        return 'far'
    elif what == 'far':
        return 'frr'
    
def optimize_models(models, what):
    if what == 'balanced':
        return list(filter(lambda x: x['far'] !=0 and x[other_what('far')] != 1, sorted(models, key=lambda x: (x['far'], x['frr']))))
    return list(filter(lambda x: x[what] !=0 and x[other_what(what)] != 1, sorted(models, key=lambda x: x[what])))

In [1373]:
def optimize_user(models, what='far', metric='log'):
    best_models = optimize_models(models, what)
    res = np.zeros(len(models[0]['res']))
    for each in best_models:
        res = res + scale_prediction(each, metric) 
    return res

In [1403]:
# RUNNING ON THE TRAINING SET
for user in users:
    user0 = User(user, dataX, dataY, kdataX, kdataY, raw_mouse_data=Authenticator.mouse_data)
    print(user0.score(np.where(optimize_user(grand_result_keyboard[user], 'balanced', metric='linear') > -3, 0, -1), user0.y_train_k))
    print(user0.score(np.where(optimize_user(grand_result_mouse[user], 'balanced', metric='linear') > -3, 0, -1), user0.y_train))

(0.0, 0.35714285714285715)
(0.02702702702702703, 0.42857142857142855)
(0.021621621621621623, 0.5714285714285714)
(0.0, 0.5714285714285714)
(0.17894736842105263, 0.2222222222222222)
(0.015789473684210527, 0.3333333333333333)
(0.021739130434782608, 0.5333333333333333)
(0.010869565217391304, 0.5333333333333333)
(0.016042780748663103, 0.5833333333333334)
(0.03208556149732621, 0.5833333333333334)
(0.04864864864864865, 0.2857142857142857)
(0.0, 0.35714285714285715)
(0.2722222222222222, 0.10526315789473684)
(0.011111111111111112, 0.47368421052631576)
(0.022222222222222223, 0.2631578947368421)
(0.0, 0.2631578947368421)
(0.05434782608695652, 0.06666666666666667)
(0.010869565217391304, 0.13333333333333333)
(0.25136612021857924, 0.25)
(0.00546448087431694, 0.6875)
(0.03910614525139665, 0.3)
(0.0111731843575419, 0.3)
(0.02717391304347826, 0.6)
(0.010869565217391304, 0.6)
(0.016483516483516484, 0.35294117647058826)
(0.0, 0.4117647058823529)


In [1416]:
# RANDOMIZED TEST SET
import random
print('Testing best models on randomly split data')

mouse_score = [0, 0]
keyboard_score = [0, 0]
combined_score = [0, 0]

    
for user in users:
    _user = User(user, dataX, dataY, kdataX, kdataY, raw_mouse_data=Authenticator.mouse_data, test_size=0.2, random_state=random.randint(0,100))
    keyboard = _user.pipeline_train_test(user_objects[user].best_models_keyboard, keyboard=1)
    mouse = _user.pipeline_train_test(user_objects[user].best_models_mouse)


    mouse = np.sum([_['res'] for _ in mouse], axis=0)

    keyboard = np.sum([_['res'] for _ in keyboard], axis=0)

    combined = mouse + keyboard

    mouse = user_objects[user].score(np.where(mouse > -3, 0, -1), user_objects[user].y_test)
    keyboard = user_objects[user].score(np.where(keyboard > -3, 0, -1), user_objects[user].y_test_k)
    combined = user_objects[user].score(np.where(combined > -5, 0, -1), user_objects[user].y_test)
    
    mouse_score[0] += mouse[0]
    mouse_score[1] += mouse[1]
    keyboard_score[0] += keyboard[0]
    keyboard_score[1] += keyboard[1]
    combined_score[0] += combined[0]
    combined_score[1] += combined[1]

    print (mouse_score, keyboard_score, combined_score, end='\n')
print('Summary : \n', 'MOUSE: ', mouse_score[0]/len(users), mouse_score[1]/len(users) ,
      '\nKEYBOARD: ', keyboard_score[0]/len(users), ', ', keyboard_score[1]/len(users), 
      '\nCOMBINED: ', combined_score[0]/len(users), ', ', keyboard_score[1]/len(users))

Testing best models on randomly split data
[0.12658227848101267, 0.8571428571428571] [0.012658227848101266, 1.0] [0.012658227848101266, 1.0]
[0.13924050632911394, 1.8571428571428572] [0.08860759493670886, 1.8571428571428572] [0.012658227848101266, 2.0]
[0.1512886991001983, 2.857142857142857] [0.5825834985511666, 1.8571428571428572] [0.060850998932438616, 3.0]
[0.4424279396065274, 3.7142857142857144] [0.645874637791673, 2.857142857142857] [0.09882568247674242, 4.0]
[0.45541495259354037, 4.714285714285714] [0.7757447676618028, 3.857142857142857] [0.13778672143778137, 5.0]
[0.5060478639859455, 5.714285714285714] [0.8643523625985117, 4.857142857142858] [0.15044494928588265, 6.0]
[0.5560478639859455, 6.714285714285714] [1.6768523625985117, 4.857142857142858] [0.22544494928588266, 7.0]
[0.669971914618857, 7.714285714285714] [1.7148270461428154, 5.857142857142858] [0.22544494928588266, 8.0]
[0.732471914618857, 8.714285714285715] [1.7898270461428154, 6.857142857142858] [0.28794494928588266, 9.

In [1354]:
optimize_models(user_objects[11].pipeline_train_test(user_objects[11].best_models_keyboard, keyboard=1), 'balanced')

[{'res': array([-1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
         -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
  'far': 0.05263157894736842,
  'frr': 0.6666666666666666,
  'config': {'method': sklearn.svm.classes.SVC,
   'params': {'kernel': 'linear', 'max_iter': 5000}},
  'index': 0},
 {'res': array([-1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
         -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
  'far': 0.05263157894736842,
  'frr': 0.6666666666666666,
 

In [1042]:
def scale_prediction(model, metric='linear'):
    if metric == 'linear':
        return model['res']
    else:
        negative_penalty = (1-model['far'])**2 
        positive_penalty = (1-model['frr'])**2
        return np.where(model['res'] == -1, -1*negative_penalty, positive_penalty)
    
#sorted(sorted(sorted(grand_result[0],key=lambda x: x['far']), key=lambda x: x['frr']), key=lambda x: x['frr']+x['far'])

In [1406]:
def scale_predictions(models):
    res = np.array([0 for _ in models[0]['res']])
    for each in models:
        res = res + scale_prediction(each)
    return res
len(users)

13

In [1511]:
data

<module 'passauth.data' from '../passauth/data.py'>

## Factors Affecting:
1. No. of minimum training size.


```15 - total_users=12
MOUSE:  0.055247286051179345 0.6953373015873016 
KEYBOARD:  0.05533615060873145 ,  0.673015873015873 
COMBINED:  0.024196920923874644 ,  0.673015873015873```

```15 total_users=12; test_size=0.4
 MOUSE:  0.07690325301052746 0.967857142857143 
KEYBOARD:  0.20926637015452634 ,  0.7797619047619048 
COMBINED:  0.03472816639327341 ,  0.7797619047619048```

```12- total_users=13; train_test_size=0.3; test_size=0.2
 MOUSE:  0.0476162196107935 0.6256410256410256 
KEYBOARD:  0.13938477484510384 ,  0.5341880341880342 
COMBINED:  0.0270226053027813 ,  0.5341880341880342```


2. Padding the data
3. No. of modeles
4. Weighing the models

In [1479]:
#total_time = 0
#for each in range(len(Authenticator.mouse_data)):
#    total_time = total_time + max(np.array(Authenticator.mouse_data[each][0])[:,2])-Authenticator.mouse_data[each][0][0][2]
timings = [max(np.array(Authenticator.mouse_data[each][0])[:,2])-Authenticator.mouse_data[each][0][0][2] 
     for each in range(len(Authenticator.mouse_data))]



In [1512]:
print('Average time per session ', sum(timings)/len(timings))
print('Shortest session ', min(timings))
print('Longest session ', max(timings))
print('Median session time ', np.median(timings))
print('Std. of session time ', np.std(timings))

Average time per session  7155.59578754615
Shortest session  319.52499999897555
Longest session  72661.1499999999
Median session time  5472.030000062659
Std. of session time  8766.772315240503
